In [1]:
from dataloader import AtariDataset
import gym
import torch.nn as nn
import torch
import numpy as np
import random
import tqdm
from tqdm import tqdm
import torch.nn.functional as F
from torch.optim import optimizer
import matplotlib.pyplot as plt
from IPython import display as ipythondisplay

## SEEDING

In [2]:
def reseed(seed):
  torch.manual_seed(seed)
  random.seed(seed)
  np.random.seed(seed)
seed = 42
reseed(seed)

## LOAD DATA

In [3]:
dataloader = AtariDataset("atari_v1")
observations, actions = dataloader.compile_data()

1


## MAKE ENVIRONMENT

In [4]:
def make_env(env_id, seed=25):
    env = gym.make(env_id, obs_type='grayscale', render_mode=None)
    env.seed(seed)
    env.action_space.seed(seed)
    env.observation_space.seed(seed)
    return env
env = make_env("SpaceInvaders-v0", seed=seed)
print(env.action_space.n)
print(env.observation_space.shape)




6
(210, 160)


A.L.E: Arcade Learning Environment (version 0.7.5+db37282)
[Powered by Stella]


## Train BC

In [5]:
from bc import SpaceInvLearner
import bc

learner = SpaceInvLearner(env)

bc.train(learner=learner, observations=observations, checkpoint_path="models/bc_learner.pth", actions=actions, num_epochs=10)

Training the learner
Training for 10 epochs


 10%|█         | 1/10 [00:00<00:04,  1.92it/s]

Epoch 0, Loss: 0.320985439909767


 20%|██        | 2/10 [00:00<00:03,  2.19it/s]

Epoch 1, Loss: 0.15203793015841832


...

 90%|█████████ | 9/10 [00:03<00:00,  2.39it/s]

Epoch 8, Loss: 0.05239081545728374


100%|██████████| 10/10 [00:04<00:00,  2.35it/s]

Epoch 9, Loss: 0.05056144293562604


SpaceInvLearner(
  (fc1): Linear(in_features=33600, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc_out): Linear(in_features=256, out_features=6, bias=True)
)

In [6]:
total_learner_reward = 0
done = False
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
obs = env.reset()
while not done:
    with torch.no_grad():
        action = learner.get_action(torch.Tensor([obs]).to(device))
    obs, reward, done, info = env.step(action)
    total_learner_reward += reward
    if done:
        break

print(total_learner_reward)

/tmp/ipykernel_29058/1640345330.py:7: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:275.)
  action = learner.get_action(torch.Tensor([obs]).to(device))


410.0


## LOAD EXPERT

In [7]:
from expert.ppo import PPOAgent, ActorCnn, CriticCnn

INPUT_SHAPE = (4, 84, 84)
ACTION_SIZE = env.action_space.n
SEED = 0
GAMMA = 0.99           # discount factor
ALPHA= 0.0001          # Actor learning rate
BETA = 0.0001          # Critic learning rate
TAU = 0.95
BATCH_SIZE = 32
PPO_EPOCH = 5
CLIP_PARAM = 0.2
UPDATE_EVERY = 1000     # how often to update the network 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
agent = PPOAgent(INPUT_SHAPE, ACTION_SIZE, SEED, device, GAMMA, ALPHA, BETA, TAU, UPDATE_EVERY, BATCH_SIZE, PPO_EPOCH, CLIP_PARAM, ActorCnn(INPUT_SHAPE, ACTION_SIZE), CriticCnn(INPUT_SHAPE))
agent.load_model("models/expert_actor.pth", device)

## DAgger Implementation

In [8]:
import dagger

dagger.interact(env, learner, agent, observations=[], actions=[], checkpoint_path="models/DAgger.pth", seed=seed, num_epochs=10)

After interaction 0, reward = 410.0
Training the learner
Training for 10 epochs


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch 0, Loss: 0.2948277564777406
Epoch 1, Loss: 0.20656341484703294


 30%|███       | 3/10 [00:00<00:00, 23.68it/s]

Epoch 2, Loss: 0.17200607228675652
Epoch 3, Loss: 0.15425679893107028
Epoch 4, Loss: 0.1446160303269999


...

 50%|█████     | 5/10 [00:03<00:03,  1.51it/s]

Epoch 4, Loss: 0.10355867628893985


 60%|██████    | 6/10 [00:04<00:02,  1.47it/s]

Epoch 5, Loss: 0.1022490273345137


KeyboardInterrupt: 